<a href="https://colab.research.google.com/github/RUND0WN75/CSCI158-Applied-Biometric-Security-FRESNO-STATE/blob/main/Preprocessing_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the Libraries for preprocessing.

In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Dowload Dataset
#create data loader
# build model
# train
# save trained model

BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

Setting up the classes for forward feeding and downloading datasets

In [ ]:
class FeedForwardnet(nn.Module):
    def __init__(self): #constructor
        super().__init__()
        self.flatten= nn.Flatten() #john is just an arbitrary name
        self.dense_layers = nn.Sequential( #allows to pack together multiple layers for sequentially
            nn.Linear(28*28, 256), #input and output features
            nn.ReLU(),
            nn.Linear(256, 10), #10 is the number of data sets
        )
        self.softmax = nn.Softmax(dim=1)

    #specifies the data flow
    def forward(self, input_data): #how to manipulated the data
        flattened_data = self.flatten(input_data)
        logits = self.dense_layers(flattened_data) #outputs
        predictions = self.softmax(logits)
        return predictions


def download_dataset():
    train_data = datasets.MNIST(
        root= "data",
        download= True,
        train= True,
        transform= ToTensor()     #takes image in ans shapes tensor
    )

    validation_data = datasets.MNIST(
        root= "data",
        download= True,
        train= True,
        transform= ToTensor()     #takes image in ans shapes tensor
    )

    return train_data, validation_data

def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
    for inputs, targets in data_loader:
        inputs, targets = inputs.to(device), targets.to(device)


        #calculate loss
        predictions = model(inputs)
        loss = loss_fn(predictions, targets)

        #back propogate the loss and update the weights
        optimiser.zero_grad() #resets gradients to zero
        loss.backward()
        optimiser.step()
    print(f"Loss: {loss.item()}")

def train(model, data_loader, loss_fn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epochs{i+1}")
        train_one_epoch(model, data_loader, loss_fn, optimiser, device)
        print("-----------------")
    print("Training is done.")

The main function and how it creates, builds and instantiates the model. While also calculating for loss of entropy.

In [12]:
if __name__ == "__main__":
    train_data, _ = download_dataset()
    print("MNIIST dataset downloaded")

    #creating a data loader for the train set
    train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE)

    #build a model
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device} device")

    feed_foward_net = FeedForwardnet().to(device) #device is a string called CUDA (GPU)

    #Instantiate loss and opt
    loss_fn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(feed_foward_net.parameters(),lr= LEARNING_RATE)
    #train the model
    train(feed_foward_net, train_data_loader, loss_fn, optimiser, device, EPOCHS)

100%|██████████| 9912422/9912422 [00:00<00:00, 112879626.75it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 46970024.75it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28820657.34it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10861190.86it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

MNIIST dataset downloaded
Using cpu device
Epochs1
Loss: 1.513471007347107
-----------------
Epochs2
Loss: 1.4988842010498047
-----------------
Epochs3
Loss: 1.4956494569778442
-----------------
Epochs4
Loss: 1.48817777633667
-----------------
Epochs5
Loss: 1.4794567823410034
-----------------
Epochs6
Loss: 1.4746742248535156
-----------------
Epochs7
Loss: 1.472772240638733
-----------------
Epochs8
Loss: 1.4730957746505737
-----------------
Epochs9
Loss: 1.4742625951766968
-----------------
Epochs10
Loss: 1.4752616882324219
-----------------
Training is done.
